7.
Study the implementation in Recommender Systems in Python 101 | Kaggle  
https://www.kaggle.com/code/gspmoreira/recommender-systems-in-python-101

where the content based filtering is built using tf-idf vector of the message content https://www.kaggle.com/code/gspmoreira/recommender-systems-in-python-101?scriptVersionId=24758977&cellId=35
replace the tf-idf by a simple sentiment analyzer result using vader

Suggest a script that implements this paradigm and perform its evaluation using Recall@N and NDCG@N metrics.

In [ ]:
#####
####  Task 7
#### "7.	Study the implementation in Recommender Systems in Python 101 | Kaggle
#### where the content based filtering is built using tf-idf vector of the
#### message content, we want to replace the tf-idf by a simple
#### sentiment analyzer result using vader. Suggest a script that
#### implements this paradigm and perform its evaluation
#### using Recall@N and NDCG@N metrics."
#####

# Initiate necessary tools
# (including vader)
# Connect with data

# determine positive/ neutral/ negative ratio for ARTICLE_TEXT entries and save this new data for later use
##### ( vader can give us evaluations like  {'neg': 0.035, 'neu': 0.877, 'pos': 0.087, 'compound': 0.9611}
##### based on what words are featured prominently in the text. Is it mostly negative, neutral or positive?
##### Details on the scoring: https://github.com/cjhutto/vaderSentiment#about-the-scoring )

# Might want to only process articles that have at least two different users
# interacting with them, otherwise there are no links between some of the data


# FUNC1 (articleID1, articleID2)
# Create a function that uses the evaluated ARTICLE_TEXT
# takes 2 article IDs as input, and outputs the distance between them
# based on....
 # combined normalized euclidean distance for the neg/neu/pos values
 # (each values should be as close to one another as possible) ?

# FUNC2 (articleID, num_of_results)
# Create function that takes 1 article ID as input and returns N of the most similar articles
# based on smallest distance counted in func1
# --> similar style of expression in ARTICLE_TEXT entries can be pointed out
#     and we can inspect if people are likely to interact with entries
#     that would be grouped together based on the vader evaluation scores.

# Evaluating the recommender system with Recall@N and NDCG@N

# Recall@N = Number of relevant items in top N recommendations / Total number of relevant items
# We declare an article relevant to a user if they have interacted with the article in any way ( = known to be interested )
# So for each user that has interacted with articles at least N+1 times, we can
# get list of the articles they interacted with,
# and for each item :
#   look up N recommended articles
#   then verify if any of those recommended articles are also ones that the same user has interacted with.
# Number of relevant items in top N recommendations per user's interacted article / Total number of articles interacted by user = Recall@N


# NDCG@N
# ? https://gist.github.com/bwhite/3726239  (has some resource links)




In [15]:

# initiate necessary tools / import panda etc (same as for earlier tasks can be used)
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import wordnet as wn
from nltk import edit_distance as ed
from scipy.spatial.distance import cosine
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
# %pip install fuzzywuzzy  #colab requires install
# from fuzzywuzzy import fuzz
# from fuzzywuzzy import process
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
from gensim.models import FastText
import torch
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
import pandas as pd


In [14]:
# Connect with data (same as earlier tasks can be used)
##articles = pd.read_csv("shared_articles.csv", delimiter= ',')
##interactions = pd.read_csv("users_interactions.csv", delimiter= ',')

articles_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/nlp project/shared_articles.csv')
articles_df = articles_df[articles_df['eventType'] == 'CONTENT SHARED']
articles_df.head(5)

interactions_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/nlp project/users_interactions.csv')
interactions_df.head(5)

comments_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/nlp project/comments.csv')
comments_df.head(5)

,personId,contentId,userReview,generatedScore,comment
0,4340306774493623681,-2261845840076552387,2.9,2,I am so angry that i made this post available...
1,-1032019229384696495,-4205346868684833897,7.5,5,No real complaints the hotel was great great ...
2,-1032019229384696495,-4205346868684833897,7.1,5,Location was good and staff were ok It is cut...
3,4340306774493623681,-2261845840076552387,3.8,2,Great location in nice surroundings the bar a...
4,-108842214936804958,5688279681867464747,6.7,3,Amazing location and building Romantic settin...


In [16]:
# additionally, import vader
%pip install vaderSentiment #colab requires install
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
#Note: If you use either the dataset or any of the VADER sentiment analysis tools (VADER sentiment lexicon or Python code for rule-based sentiment analysis engine) in your research, please cite the above paper. For example:
#
#    Hutto, C.J. & Gilbert, E.E. (2014). VADER: A Parsimonious Rule-based Model for Sentiment Analysis of Social Media Text. Eighth International Conference on Weblogs and Social Media (ICWSM-14). Ann Arbor, MI, June 2014.


In [37]:
#vader example
sentences = ["VADER is smart, handsome, and funny.",  # positive sentence example
             "VADER is smart, handsome, and funny!",  # punctuation emphasis handled correctly (sentiment intensity adjusted)
             "VADER is very smart, handsome, and funny.", # booster words handled correctly (sentiment intensity adjusted)
             "VADER is VERY SMART, handsome, and FUNNY.",  # emphasis for ALLCAPS handled
             "VADER is VERY SMART, handsome, and FUNNY!!!", # combination of signals - VADER appropriately adjusts intensity
             "VADER is VERY SMART, uber handsome, and FRIGGIN FUNNY!!!", # booster words & punctuation make this close to ceiling for score
             "VADER is not smart, handsome, nor funny.",  # negation sentence example
             "The book was good.",  # positive sentence
             ]
analyzer = SentimentIntensityAnalyzer()
vs_articles = None

for sentence in sentences:
    vaderscore = analyzer.polarity_scores(sentence)
    print("{:-<70} {}".format(sentence, str(vaderscore)))
    vs_articles = pd.concat([vs_articles, pd.DataFrame(vaderscore, index=[0])])

vs_articles

VADER is smart, handsome, and funny.---------------------------------- {'neg': 0.0, 'neu': 0.254, 'pos': 0.746, 'compound': 0.8316}
VADER is smart, handsome, and funny!---------------------------------- {'neg': 0.0, 'neu': 0.248, 'pos': 0.752, 'compound': 0.8439}
VADER is very smart, handsome, and funny.----------------------------- {'neg': 0.0, 'neu': 0.299, 'pos': 0.701, 'compound': 0.8545}
VADER is VERY SMART, handsome, and FUNNY.----------------------------- {'neg': 0.0, 'neu': 0.246, 'pos': 0.754, 'compound': 0.9227}
VADER is VERY SMART, handsome, and FUNNY!!!--------------------------- {'neg': 0.0, 'neu': 0.233, 'pos': 0.767, 'compound': 0.9342}
VADER is VERY SMART, uber handsome, and FRIGGIN FUNNY!!!-------------- {'neg': 0.0, 'neu': 0.294, 'pos': 0.706, 'compound': 0.9469}
VADER is not smart, handsome, nor funny.------------------------------ {'neg': 0.646, 'neu': 0.354, 'pos': 0.0, 'compound': -0.7424}
The book was good.---------------------------------------------------- {'ne

,neg,neu,pos,compound
0,0.000,0.254,0.746,0.8316
0,0.000,0.248,0.752,0.8439
0,0.000,0.299,0.701,0.8545
0,0.000,0.246,0.754,0.9227
0,0.000,0.233,0.767,0.9342
0,0.000,0.294,0.706,0.9469
0,0.646,0.354,0.000,-0.7424
0,0.000,0.508,0.492,0.4404


In [ ]:
# determine positive/ neutral/ negative ratio for article text

# Note we might want to only process articles that have at least N users
# or users with more than 1 article interacted
# otherwise there are no links between some of the data

# we have original data in articles_df , interactions_df, comments_df

In [17]:
# leave out articles in languages other than english
# (just take first 100 for speedy testing)
articles_en = articles_df[articles_df['lang'] == 'en'].head(100)

In [18]:
analyzer = SentimentIntensityAnalyzer()
# get text, turn into a list of sentiment scores. save to use as list of entries
vs_neg = []
vs_neu = []
vs_pos = []
vs_compound = []
for entry in articles_en['text']:
  vaderscore = analyzer.polarity_scores(entry)
  vs_neg.append(vaderscore['neg'])
  vs_neu.append(vaderscore['neu'])
  vs_pos.append(vaderscore['pos'])
  vs_compound.append(vaderscore['compound'])

In [19]:
# add vader numbers to the dataset as column
articles_en['neg'] = vs_neg
articles_en['neu'] = vs_neu
articles_en['pos'] = vs_pos
articles_en['compound'] = vs_compound

# check see if the new columns showed up
articles_en.head(10)

In [38]:
#example ids
articleID1 = -4110354420726924665
articleID2 = 5191381587333696286

score1 = analyzer.polarity_scores( articles_en[articles_en['contentId'] == articleID1]['text'] )
score2 = analyzer.polarity_scores( articles_en[articles_en['contentId'] == articleID2]['text'] )

#use score as vector
print(score1)
score1 = list(score1.values())
print(score1)

score2 = np.array(list(score2.values()))
print(score2)

distance = np.linalg.norm(score1 - score2)
print(distance)


{'neg': 0.026, 'neu': 0.824, 'pos': 0.15, 'compound': 0.9993}
[0.026, 0.824, 0.15, 0.9993]
[0.016  0.848  0.136  0.9968]
0.02963528302547489


In [ ]:
### FUNC1 (articleID1, articleID2)
# Create a function that uses the evaluated ARTICLE_TEXT
# takes 2 article IDs as input, and outputs the distance between them
# based on....
 # combined normalized euclidean distance for the neg/neu/pos values
 # (each values should be as close to one another as possible) ?
score1 = None
score2 = None

#example ids
articleID1 = -4110354420726924665
articleID2 = 5191381587333696286

score1 = articles_en[articles_en['contentId'] == articleID1]
score1 = np.array((score1['neg'], score1['neu'], score1['pos'], score1['compound']))

score2 = articles_en[articles_en['contentId'] == articleID2]
score2 = [score2['neg'],score2['neu'],score2['pos'],score2['compound']]

distance = np.linalg.norm(score1 - score2)
print(distance)


0.02963528302547489


In [31]:
import math

### FUNC1 (articleID1, articleID2)
# Create a function that uses the evaluated ARTICLE_TEXT
# takes 2 article IDs as input, and outputs the distance between them
def vs_distance(articleID1, articleID2):
  print(articleID1,articleID2)

  distance=None
  score1=None
  score2=None

  score1 = articles_en[articles_en['contentId'] == articleID1]
  score1 = np.array((score1['neg'], score1['neu'], score1['pos'], score1['compound']))

  score2 = articles_en[articles_en['contentId'] == articleID2]
  score2 = np.array((score2['neg'],score2['neu'],score2['pos'],score2['compound']))

  print(score1)
  print(score2)
  distance = math.dist( score1, score2 )

  print("returnt  ", distance)
  return distance

In [35]:
print(vs_distance(-4110354420726924665, 5191381587333696286))

-4110354420726924665 5191381587333696286
[[0.026 ]
 [0.824 ]
 [0.15  ]
 [0.9993]]
[[0.016 ]
 [0.848 ]
 [0.136 ]
 [0.9968]]
returnt   0.02963528302547489
0.02963528302547489


In [34]:
vs_distance(-4110354420726924665, 5191381587333696286)

# FUNC2 (articleID, num_of_results)
# Create function that takes 1 article ID as input and returns N of the most similar articles
# based on smallest distance counted in func1

# form list of ID + similarity score

eid1 = '5191381587333696286'
distancelist = []

for entry in articles_en['contentId']:
  print(eid1)
  print(entry)
  returnedvl = vs_distance2(eid1, entry)
  print(returnedvl)
  distancelist.append( returnedvl )


-4110354420726924665 5191381587333696286
[[0.026 ]
 [0.824 ]
 [0.15  ]
 [0.9993]]
[[0.016 ]
 [0.848 ]
 [0.136 ]
 [0.9968]]
returnt   0.02963528302547489
5191381587333696286
-4110354420726924665
5191381587333696286 -4110354420726924665
[]
[[0.026 ]
 [0.824 ]
 [0.15  ]
 [0.9993]]


TypeError: ignored

TASK 8
repeat 7) when the content of message is constituted by a vector corresponding to the concatenation of [Positive Sentiment score, Negative Sentiment Score, Length of the message, Ratio of stopwords /uncommon characters, Number of Personal Pronouns].

In [ ]:
## Task 8
# vector corresponding to the concatenation of
 # Positive Sentiment score (vader)
 # Negative Sentiment Score (vader)
 # Length of the message
 # Ratio of stopwords / uncommon characters
 # Number of Personal Pronouns
